In [1]:
import pandas as pd
import numpy  as np
import torch
import torch.nn.functional as Fun

d:\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_excel('completeData_autotuneVIM_KNN_method.xlsx')

In [3]:
df=df[['CAREER_SHIFTS', 
           'WEEKEND_SHIFTS', 
           'CAREER_6_MONTH_DELIVERED_AVERAGE', 
           'CAREER_RETURNS', 
           'CAREER_DAILY_RETURN_AVERAGE', 
           'RETURNS_90_DAYS', 
           'PEICES_DELIVERED_90_DAYS', 
           'CAREER_PEICES_DELIVERED', 
           'CAREER_HOURS_WORKED', 
           'CAREER_HOURS_WORKED_DAILY_AVERAGE', 
           'SIX_MONTH_DAILY_RETURN_AVERAGE', 
           'WEEK_SHIFTS', 
           'CAREER_DAILY_DELIVERED_AVERAGE', 
           'OVERNIGHT_SHIFTS', 
           'DAILY_DELIVERY_PAST_MONTH_AVERAGE', 
           'OVERNIGHT_SHIFTS_PER_WEEK_SIX_MONTH_AVERAGE', 
           'OVERNIGHT_SHIFTS_PER_WEEK_LAST_MONTH_AVERAGE','STATUS']]

In [4]:
df['STATUS']=df['STATUS'].apply(lambda x:0 if x=='TERMINATED EMPLOYEE'else 1 )

In [5]:
df

,CAREER_SHIFTS,WEEKEND_SHIFTS,CAREER_6_MONTH_DELIVERED_AVERAGE,CAREER_RETURNS,CAREER_DAILY_RETURN_AVERAGE,RETURNS_90_DAYS,PEICES_DELIVERED_90_DAYS,CAREER_PEICES_DELIVERED,CAREER_HOURS_WORKED,CAREER_HOURS_WORKED_DAILY_AVERAGE,SIX_MONTH_DAILY_RETURN_AVERAGE,WEEK_SHIFTS,CAREER_DAILY_DELIVERED_AVERAGE,OVERNIGHT_SHIFTS,DAILY_DELIVERY_PAST_MONTH_AVERAGE,OVERNIGHT_SHIFTS_PER_WEEK_SIX_MONTH_AVERAGE,OVERNIGHT_SHIFTS_PER_WEEK_LAST_MONTH_AVERAGE,STATUS
0,2,154,152.400000,206,14.700000,2,1054,4638,0,0.000000,6.600000,2,136.411765,1,533.000000,4.000000,3.666667,1
1,1077,211,338.636364,784,15.285714,126,26760,50812,11659,10.785383,10.714286,2,396.650406,29,15484.000000,0.000000,0.000000,1
2,798,147,1079.405405,553,11.529412,169,97884,142302,8485,10.566625,11.500000,1,1185.850000,8,20432.000000,0.000000,0.000000,1
3,855,211,670.661290,667,11.500000,142,54264,88736,9845,11.421113,11.750000,2,657.303704,26,16420.000000,0.035714,0.166667,1
4,751,187,648.127660,859,12.782609,226,38528,64204,8230,10.915119,11.263158,2,635.683168,62,24634.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,655,257,1013.384615,169,7.700000,79,41586,42,7098,10.836641,7.125000,2,42.000000,23,9666.000000,0.000000,0.000000,0
890,242,34,470.000000,228,12.636364,22,8886,34216,2424,9.813765,13.200000,2,488.800000,100,2468.666667,0.000000,0.000000,1
891,1060,253,577.238806,846,14.181818,152,49086,80798,11873,11.137898,10.461538,2,561.097222,25,18052.000000,0.000000,0.000000,1
892,876,239,401.277108,657,8.500000,129,17829,134246,8716,9.893303,8.071429,2,1475.230769,79,8069.500000,0.035714,0.166667,1


In [6]:
class BPNetModel(torch.nn.Module):
    def __init__(self,n_feature,n_hidden,n_output):
        super(BPNetModel, self).__init__()
        self.hiddden=torch.nn.Linear(n_feature,n_hidden)
        self.out=torch.nn.Linear(n_hidden,n_output)
    def forward(self,x):
        x=Fun.relu(self.hiddden(x)) 
        out=Fun.softmax(self.out(x),dim=1) 
        return out


In [7]:
net=BPNetModel(n_feature=17,n_hidden=10,n_output=2) #
optimizer=torch.optim.Adam(net.parameters(),lr=0.02) 
loss_fun=torch.nn.CrossEntropyLoss()

In [8]:
from sklearn.model_selection import train_test_split
x=torch.FloatTensor(np.array(df.iloc[:,0:17]))
y=torch.LongTensor(df.iloc[:,17])
x_train,x_test,y_train,y_test=train_test_split(x,y)



In [9]:
epochs=20
loss_steps=np.zeros(epochs) 
accuracy_steps=np.zeros(epochs)
for epoch in range(epochs):
    y_pred=net(x_train) 
    loss=loss_fun(y_pred,y_train)
    optimizer.zero_grad() 
    loss.backward() 
    optimizer.step() 
    loss_steps[epoch]=loss.item()
    running_loss = loss.item()
    print(f"第{epoch}次训练,loss={running_loss}".format(epoch,running_loss))
    with torch.no_grad(): 
        y_pred=net(x_test)
        correct=(torch.argmax(y_pred,dim=1)==y_test).type(torch.FloatTensor)
        accuracy_steps[epoch]=correct.mean()
        print("预测准确率", accuracy_steps[epoch])

第0次训练,loss=0.5040386319160461
预测准确率 0.8035714030265808
第1次训练,loss=0.5276092290878296
预测准确率 0.8258928656578064
第2次训练,loss=0.501075029373169
预测准确率 0.7946428656578064
第3次训练,loss=0.507294774055481
预测准确率 0.7723214030265808
第4次训练,loss=0.5392969250679016
预测准确率 0.8125
第5次训练,loss=0.5087844133377075
预测准确率 0.8080357313156128
第6次训练,loss=0.5087839365005493
预测准确率 0.8125
第7次训练,loss=0.5087839365005493
预测准确率 0.8080357313156128
第8次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第9次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第10次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第11次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第12次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第13次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第14次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第15次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第16次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第17次训练,loss=0.5102764964103699
预测准确率 0.8080357313156128
第18次训练,loss=0.51027649641036